## Задача 2

Яндекс.Еда осуществляет доставку еды из ресторанов. При этом у каждого ресторана есть зона, в рамках которой осуществляется доставка. 

Зона представляет собой полигон (заданы координаты его вершин). Пользователь в зависимости от своего местоположения (координат точки) видит разное количество доступных ресторанов. 

Нам важно, чтобы у каждого пользователя было достаточное количество ресторанов для выбора. Задача заключается в том, чтобы для каждого пользователя посчитать доступное ему количество ресторанов.

Использовать Python (результат .py или .ipynb файл).

Данные, которые есть (для простоты в формате .csv, несколько первых строк):   
**user_coordinates.csv** (примерно 300 тыс. строк, user_id – идентификатор пользователя)  
**place_zone_coordinates.csv** (примерно 500 тыс. строк,
place_id – идентификатор ресторана, 
point_number – порядковый номер вершины полигона)

Формат результата:

id,number_of_places_available  
1,2  
2,19  
3,0  

### Решение

In [75]:
import itertools
import bisect

user_coordinates = 'user_coordinates.csv'
place_zone_coordinates = 'place_zone_coordinates.csv'

In [98]:
# Итератор для файла пользователями

class UserIterator:
    """
    Iterator class for iterate over users in csv file
    """
    def __iter__(self):
        return self

    def __init__(self, filename):
        # file headings: user_id,loc_lat,loc_lon
        
        self.filename = filename
        self.file = open(filename)
        self.headings = self.file.readline().split(',')
        
        print("INFO: read user file with headings:", *self.headings)
        # self.counter = 0

    def __next__(self):
        line = self.file.readline()
        if not line:
            raise StopIteration
        line = list(map(float, line.split(",")))
        return int(line[0]), tuple(line[1:])

test_user_iterator = UserIterator(user_coordinates)

for user_id, user_point in test_user_iterator:
    print("User: ", user_id, user_point)

INFO: read user file with headings: user_id loc_lat loc_lon

User:  1 (55.737564, 37.345186)
User:  2 (56.234564, 37.23459)
User:  3 (55.234578, 36.295745)


In [99]:
# Итератор для файла с зонами доставки
    
class PolygonIterator:
    """
    Iterator class for iterate over places in csv file
    """
    
    def __iter__(self):
        return self

    def __init__(self, filename):
        
        self.filename = filename
        self.file = open(filename)
        self.headings = self.file.readline().split(',')
        
        place_id, point = self.read_point()
        self.current_place_id = place_id
        self.current_point = point
        
        print("INFO: read file with headings:", *self.headings)
        self.counter = 0
    
    def read_point(self):
        line = self.file.readline()
        if not line:
            return None, None
        point = list(map(float, line.split(',')))
        place_id = point[0]
        point = tuple(point[1:3])
        return int(place_id), point

    def __next__(self):
        
        ## Init polygon points
        if not self.current_place_id:
            raise StopIteration
            
        place_id = self.current_place_id
        points = list()
        points.append(self.current_point)
        
        
        while place_id == self.current_place_id:

            place_id, point = self.read_point()
            
            if place_id != self.current_place_id:
                continue
            
            points.append(point)
        
        else:
            prev_place_id = self.current_place_id
            self.current_place_id = place_id
            self.current_point = point
            return prev_place_id, points
            
test_polygon_iterator = PolygonIterator(place_zone_coordinates)

for polygon_id, polygon_points in test_polygon_iterator:
    print("Polygon: ", polygon_id, polygon_points)

INFO: read file with headings: place_id loc_lat loc_lon point_number

Polygon:  1 [(55.747022, 37.787073), (55.751713, 37.784328), (55.753878, 37.777638), (55.751031, 37.779351)]
Polygon:  2 [(55.803885, 37.458311), (55.808677, 37.464054), (55.809763, 37.461314), (55.81084, 37.458654)]


In [100]:
# Функция для проверки находится ли точка внутри полигона

def is_in_delivery_area(point: list, poly: list) -> bool:
    """
    Args:
    point : list of int (x: latitude,  y: longitude)
    polygon: list of polygon points
    
    Returns:
        1 if the point is in the path
        0 if the point isn't in the path
    """
    x = point[0]
    y = point[1]
    num = len(poly)
    i = 0
    j = num - 1
    c = False
    for i in range(num):
        if ((poly[i][1] > y) != (poly[j][1] > y)) and \
                (x < poly[i][0] + (poly[j][0] - poly[i][0]) * (y - poly[i][1]) /
                                  (poly[j][1] - poly[i][1])):
            c = not c
        j = i
    return int(c)

In [102]:
# Реализация алгоритма со сложностью О(MNK) 
# M - количество пользователей, 
# N - количество ресторанов, 
# K - количество точек полигона

import csv 

results = list([0, 0])
iter_place_zone = PolygonIterator('place_zone_coordinates.csv')

for place_id, place_points in iter_place_zone:
    iter_users = UserIterator('user_coordinates.csv')
    
    print("Polygon: ", place_id, place_points)
    for user_id, user_point in iter_users:
        print("User: ", user_id, user_point)
        result = is_in_delivery_area(user_point, place_points)
        if len(results) < user_id:
            results.append(result)
        else:
            results[user_id] += result
    

INFO: read file with headings: place_id loc_lat loc_lon point_number

INFO: read user file with headings: user_id loc_lat loc_lon

Polygon:  1 [(55.747022, 37.787073), (55.751713, 37.784328), (55.753878, 37.777638), (55.751031, 37.779351)]
User:  1 (55.737564, 37.345186)
User:  2 (56.234564, 37.23459)


IndexError: list index out of range

In [ ]:
# Write results to file
        
        
with open("result.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(mylist)
        
        